In [1]:
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime
import pandas as pd

# Define start and end dates
start_date = "2010-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')
# Step 1: Fetch NASDAQ data using QQQ ETF as a proxy
try:
    nasdaq_data = yf.download("NDAQ", start=start_date, end=end_date)[["Open", "High", "Low", "Close", "Volume"]]
    print("NASDAQ data sample:")
    print(nasdaq_data.head())
except Exception as e:
    print(f"Error retrieving NASDAQ data: {e}")
    nasdaq_data = pd.DataFrame()  # Empty DataFrame if not available

# Step 2: Fetch economic indicators from FRED
indicators = {
    "InterestRate": "FEDFUNDS",      # Federal Funds Rate
    "ExchangeRate": "DEXUSEU",       # USD to EUR Exchange Rate
    "VIX": "VIXCLS",                 # Volatility Index
    "TEDSpread": "TEDRATE",          # TED Spread
    "EFFR": "EFFR"                   # Effective Federal Funds Rate
}

# Retrieve each indicator and store in a dictionary
econ_data = {}
for label, fred_code in indicators.items():
    try:
        data = pdr.get_data_fred(fred_code, start=start_date, end=end_date)
        print(f"{label} data sample:")
        print(data.head())
        econ_data[label] = data
    except Exception as e:
        print(f"Could not retrieve {label} from FRED: {e}")
        econ_data[label] = pd.DataFrame()  # Empty DataFrame if not available

# Step 3: Fetch Gold and Oil prices using Yahoo Finance
try:
    gold_data = yf.download("GC=F", start=start_date, end=end_date)[["Close"]].rename(columns={"Close": "Gold"})
    print("Gold data sample:")
    print(gold_data.head())
except Exception as e:
    print(f"Error retrieving Gold data: {e}")
    gold_data = pd.DataFrame()

try:
    oil_data = yf.download("CL=F", start=start_date, end=end_date)[["Close"]].rename(columns={"Close": "Oil"})
    print("Oil data sample:")
    print(oil_data.head())
except Exception as e:
    print(f"Error retrieving Oil data: {e}")
    oil_data = pd.DataFrame()

# Step 4: Combine all datasets
# Convert economic indicators dictionary to a DataFrame, excluding any that failed
econ_df = pd.concat([df for df in econ_data.values() if not df.empty], axis=1)
econ_df.columns = [label for label, df in econ_data.items() if not df.empty]

# Combine all data into a single DataFrame
combined_data = nasdaq_data.join(econ_df, how='outer')
combined_data = combined_data.join(gold_data, how='outer')
combined_data = combined_data.join(oil_data, how='outer')

# Fill missing values and save to CSV if any data exists
if not combined_data.empty:
    combined_data = combined_data.fillna(method='ffill').dropna()
    print("Data has been successfully saved to 'nasdaq_combined_data.csv'.")
else:
    print("No data to save. Check each data source for availability.")


[*********************100%***********************]  1 of 1 completed


NASDAQ data sample:
                Open      High       Low     Close   Volume
Date                                                       
2010-01-04  6.640000  6.810000  6.633333  6.746667  6514500
2010-01-05  6.643333  6.773333  6.643333  6.766667  4445100
2010-01-06  6.733333  6.786667  6.720000  6.763333  7340100
2010-01-07  6.750000  6.766667  6.630000  6.673333  8498400
2010-01-08  6.676667  6.766667  6.626667  6.743333  4347600
InterestRate data sample:
            FEDFUNDS
DATE                
2010-01-01      0.11
2010-02-01      0.13
2010-03-01      0.16
2010-04-01      0.20
2010-05-01      0.20
ExchangeRate data sample:
            DEXUSEU
DATE               
2010-01-01      NaN
2010-01-04   1.4419
2010-01-05   1.4402
2010-01-06   1.4404
2010-01-07   1.4314
VIX data sample:
            VIXCLS
DATE              
2010-01-01     NaN
2010-01-04   20.04
2010-01-05   19.35
2010-01-06   19.16
2010-01-07   19.06
TEDSpread data sample:
            TEDRATE
DATE               
2010-01-

[*********************100%***********************]  1 of 1 completed


Gold data sample:
                   Gold
Date                   
2010-01-04  1117.699951
2010-01-05  1118.099976
2010-01-06  1135.900024
2010-01-07  1133.099976
2010-01-08  1138.199951


[*********************100%***********************]  1 of 1 completed

Oil data sample:
                  Oil
Date                 
2010-01-04  81.510002
2010-01-05  81.769997
2010-01-06  83.180000
2010-01-07  82.660004
2010-01-08  82.750000
Data has been successfully saved to 'nasdaq_combined_data.csv'.


In [2]:
df=combined_data

In [3]:
df = df.rename(columns={"Unnamed: 0": "Date"})


In [4]:
df.head()

,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil
2010-01-04,6.640000,6.810000,6.633333,6.746667,6514500.0,0.11,1.4419,20.04,0.17,0.12,1117.699951,81.510002
2010-01-05,6.643333,6.773333,6.643333,6.766667,4445100.0,0.11,1.4402,19.35,0.18,0.12,1118.099976,81.769997
2010-01-06,6.733333,6.786667,6.720000,6.763333,7340100.0,0.11,1.4404,19.16,0.19,0.12,1135.900024,83.180000
2010-01-07,6.750000,6.766667,6.630000,6.673333,8498400.0,0.11,1.4314,19.06,0.20,0.10,1133.099976,82.660004
2010-01-08,6.676667,6.766667,6.626667,6.743333,4347600.0,0.11,1.4357,18.13,0.20,0.11,1138.199951,82.750000


In [5]:
df.tail()

,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil
2024-11-12,79.940002,79.940002,79.019997,79.129997,2400700.0,4.83,1.0598,14.71,0.09,4.58,2600.000000,68.120003
2024-11-13,79.500000,80.349998,79.400002,79.870003,1861600.0,4.83,1.0567,14.02,0.09,4.58,2580.800049,68.430000
2024-11-14,80.480003,80.769997,78.000000,78.059998,3188400.0,4.83,1.0562,14.31,0.09,4.58,2568.199951,68.699997
2024-11-15,78.230003,78.930000,78.160004,78.760002,4685200.0,4.83,1.0552,16.14,0.09,4.58,2565.699951,67.019997
2024-11-18,78.360001,79.889999,78.360001,79.779999,1865600.0,4.83,1.0552,16.14,0.09,4.58,2610.600098,69.160004


In [6]:
df.to_csv("nasdq.csv")

In [7]:
df.isnull().sum()

Open            0
High            0
Low             0
Close           0
Volume          0
InterestRate    0
ExchangeRate    0
VIX             0
TEDSpread       0
EFFR            0
Gold            0
Oil             0
dtype: int64

In [8]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'InterestRate',
       'ExchangeRate', 'VIX', 'TEDSpread', 'EFFR', 'Gold', 'Oil'],
      dtype='object')